In [1]:
import numpy as np
import torchvision.transforms as transforms
import torch
import torchvision.models as models

from src.extractor_utils import (predict, save_prediction)
from src.Dataset import KaggleSafeDriverDataset
from src.imgnet_utils import denormalize
from src.data_loader import _create_dataLoader
from src.data_loader import _create_dataLoader
from src.convnet_models import (MyResNet, MyInception, MyDenseNet,MyVgg16Net)

# Choosing resources to extrac the features

In [2]:
print("{} GPU's available:".format(torch.cuda.device_count()) )
cpu_count = torch.multiprocessing.cpu_count()
print("\ncpu_count: {}".format(cpu_count))

1 GPU's available:

cpu_count: 8


In [3]:
use_gpu = True
use_DataParalel= False
use_CPU= False        

if use_gpu:
    if use_DataParalel: 
        print("Using DataParalel in all {} GPUS".format(torch.cuda.device_count()))

    else:
        print('Using only one GPU')

if use_CPU:         # http://pytorch.org/tutorials/beginner/former_torchies/parallelism_tutorial.html 
    print("Using {} CPU's".format(cpu_count))

Using only one GPU


# Path to raw data

In [4]:
path2train = "/home/Prashanth/dl4cv-project/data/train" 
path2test = "/home/Prashanth/dl4cv-project/data/test" 

# Image transformation

In [5]:
imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std  = np.array([0.229, 0.224, 0.225])

img_width = img_height=224 #to use InceptionV3 it must img_width and img_height be changed to 300

# Data augmentation and normalization for training 
data_transforms = {
    'train': transforms.Compose([
        transforms.Scale((img_width, img_height)),
        transforms.ToTensor(),
        transforms.Normalize(imagenet_mean, imagenet_std),
    ]),
    'valid': transforms.Compose([
        transforms.Scale((img_width, img_height)),
        transforms.ToTensor(),
        transforms.Normalize(imagenet_mean, imagenet_std),
    ]),
}        

/home/Prashanth/anaconda3/lib/python3.5/site-packages/torchvision/transforms/transforms.py:156: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


# Creating Dataset & DataLoader

In [6]:
batch_size = 32
use_only = 1.0 # Use only is the percentage of the full dataset

In [7]:
dsets = {
    'train': KaggleSafeDriverDataset(path2train, transforms=data_transforms['train'],use_only=use_only),
    'valid': KaggleSafeDriverDataset(path2train, transforms=data_transforms['valid'],use_only=use_only, is_val=True, val_size=0.2),
    'test':  KaggleSafeDriverDataset(path2test, transforms=data_transforms['valid'],use_only=use_only, is_test=True),
}

In [8]:
dset_loaders = _create_dataLoader(dsets, batch_size, pin_memory=False, use_shuffle= True)

In [9]:
dset_sizes = {x: len(dsets[x]) for x in ['train','valid', 'test']} 
dset_classes = len(dsets['train'].y)
dset_classes, dset_sizes

# Dataset have much more samples than datatrain ***It comes from the test.zip****

(17940, {'test': 79726, 'train': 17940, 'valid': 4484})

# Checking GPU usage

In [10]:
if use_gpu:
    !free -h
    !nvidia-smi

              total        used        free      shared  buff/cache   available
Mem:            29G        2.4G         16G         12M         10G         26G
Swap:            0B          0B          0B
Sat Feb  3 23:02:29 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 390.12                 Driver Version: 390.12                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    71W / 149W |    848MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                            

# Choosing Model

In [11]:
use_resnet = False
use_inception = False
use_denseNet = False
use_vgg16 = True

if use_resnet:
    print('Using ResNet model')
    model_name = "ResNet"
#    model = MyResNet()
    model = models.resnet18()
elif use_inception:
    print('Using Inception model')
    model_name = "Inception"
    model = MyInception()
elif use_denseNet:
    print('Using DenseNet model')
    model_name = "DenseNet"    
    model = MyDenseNet()
elif use_vgg16:
    print('Using VGG16 model')
    model_name = "vgg16"    
    model = MyVgg16Net()
    print(model)
        

Using VGG16 model
MyVgg16Net(
  (mrnc): MyVggConv(
    (model): VGG(
      (features): Sequential(
        (0): Conv2d (3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace)
        (2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU(inplace)
        (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
        (5): Conv2d (64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (6): ReLU(inplace)
        (7): Conv2d (128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (8): ReLU(inplace)
        (9): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
        (10): Conv2d (128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (11): ReLU(inplace)
        (12): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (13): ReLU(inplace)
        (14): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (15): Re

In [12]:
if use_gpu:
    if use_DataParalel:
        print("Using all GPU's ")
        model = torch.nn.DataParallel(model) #device_ids=[1,3]
        model.cuda()
        convnet = model.module.mrnc
    else:
        print('Using GPU')# {}'.format(device_id))
        model.cuda()
        convnet = model.mrnc 

if use_CPU:
    print("Using CPU's")
    convnet = model.mrnc 

Using GPU


# Extracting the features

In [13]:
#extract features from images
convOutput_train = predict(dset_loaders['train'], convnet,use_gpu=use_gpu)
convOutput_valid = predict(dset_loaders['valid'], convnet,use_gpu=use_gpu)
convOutput_test = predict(dset_loaders['test'], convnet,use_gpu=use_gpu)

predict: 560/560 ok
Execution time 253.00 s
predict: 140/140 ok
Execution time 63.00 s
predict: 2491/2491 ok
Execution time 1120.00 s


In [14]:
print(convOutput_train['true'].size(), convOutput_train['pred'].size())
print(convOutput_valid['true'].size(), convOutput_valid['pred'].size())
print(convOutput_test['true'].size(), convOutput_test['pred'].size())

torch.Size([17940]) torch.Size([17940, 1000])
torch.Size([4484]) torch.Size([4484, 1000])
torch.Size([79726]) torch.Size([79726, 1000])


In [15]:
print(convOutput_train['true'].type(), convOutput_train['pred'].type())

torch.cuda.LongTensor torch.cuda.FloatTensor


# Saving features

In [16]:
sav_feats= {
    'train': (convOutput_train['pred'], convOutput_train['true'],model_name),
    'valid': (convOutput_valid['pred'], convOutput_valid['true'],model_name),
    'test': (convOutput_test['pred'], convOutput_test['true'],model_name)

}

In [17]:
sav_feats['train'][2]

'vgg16'

In [18]:
path2features = './features/'

In [19]:
save_prediction(path2features,sav_feats)


Saving vgg16 valid
Saved in:./features/valid/vgg16.npz

Saving vgg16 train
Saved in:./features/train/vgg16.npz

Saving vgg16 test
Saved in:./features/test/vgg16.npz
